it seems like you have to run the notebook locally because i cant install biogeme online

In [1]:
#pip install biogeme

In [2]:
import pandas as pd
import biogeme.database as db
from biogeme.expressions import Variable

import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta

# Read t h e d a t a
df = pd.read_csv( 'lpmc01.dat' , sep='\t' )
database = db.Database ( 'lmpc01' , df )

trip_id = Variable('trip_id') 
household_id = Variable('household_id') 
person_n = Variable('person_n') 
trip_n = Variable('trip_n') 
travel_mode = Variable('travel_mode') 
purpose = Variable('purpose') 
fueltype = Variable('fueltype') 
faretype = Variable('faretype') 
bus_scale = Variable('bus_scale') 
survey_year = Variable('survey_year') 
travel_year = Variable('travel_year') 
travel_month = Variable('travel_month') 
travel_date = Variable('travel_date') 
day_of_week = Variable('day_of_week') 
start_time = Variable('start_time') 
age = Variable('age') 
female = Variable('female') 
driving_license = Variable('driving_license') 
car_ownership = Variable('car_ownership') 
distance = Variable('distance') 
dur_walking = Variable('dur_walking') 
dur_cycling = Variable('dur_cycling') 
dur_pt_access = Variable('dur_pt_access') 
dur_pt_rail = Variable('dur_pt_rail') 
dur_pt_bus = Variable('dur_pt_bus') 
dur_pt_int = Variable('dur_pt_int') 
pt_interchanges = Variable('pt_interchanges') 
dur_driving = Variable('dur_driving') 
cost_transit = Variable('cost_transit') 
cost_driving_fuel = Variable('cost_driving_fuel') 
cost_driving_ccharge = Variable('cost_driving_ccharge') 
driving_traffic_percent = Variable('driving_traffic_percent') 
#
CHOICE = database.DefineVariable('CHOICE' , travel_mode)

#CREATE VARIABLES

cost_driving  = database.DefineVariable('cost_driving' , cost_driving_fuel + cost_driving_ccharge)
tt_driving    = database.DefineVariable('tt_driving' , dur_driving)
cost_cycling  = database.DefineVariable('cost_cycling' , dur_cycling * 0)
cost_walking  = database.DefineVariable('cost_walking' , dur_walking * 0)
tt_walking    = database.DefineVariable('tt_walking' , dur_walking)
tt_cycling    = database.DefineVariable('tt_cycling' , dur_cycling)
cost_publict  = database.DefineVariable('cost_publict' , cost_transit) 
tt_publict    = database.DefineVariable('tt_publict' , dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int  )


#pour Beta seul le dernier argument est important : 
#a flag that indicates if the parameter must be estimated (0) or if it keeps its default value (1).
B0_driving = Beta('B0_driving', 0, None, None, 0)
B0_cycling = Beta('B0_cycling', 0, None, None, 0)
B0_walking = Beta('B0_walking', 0, None, None, 1) # normalized to zero
B0_publict = Beta('B0_publict', 0, None, None, 0)
B1 = Beta('B1', 0, None, None, 0)
B2 = Beta('B2', 0, None, None, 0)


# D e f i n i t i o n  o f  t h e  u t i l i t y  f u n c t i o n s

V_driving = ( B0_driving + B1 * cost_driving + B2 * tt_driving )
V_cycling = ( B0_cycling + B1 * cost_cycling + B2 * tt_cycling )
V_publict = ( B0_walking + B1 * cost_walking + B2 * tt_walking )
V_walking = ( B0_publict + B1 * cost_publict + B2 * tt_publict )

# A s s o c i a t e  u t i l i t y  f u n c t i o n s  w i t h  t h e numbering o f  a l t e r n a t i v e s

V = {1: V_driving , 2: V_cycling , 3: V_walking , 4: V_publict }

#il faut decrire le fait que tous les choix sont available
un = database.DefineVariable('un' , cost_publict * 0 + 1) 
av = {1 : un, 2 : un, 3 : un, 4 : un }

# D e f i n i t i o n  o f  t h e model . This i s  t h e  c o n t r i b u t i o n  o f each
# o b s e r v a t i o n  t o  t h e  l o g l i k e l i h o o d  f u n c t i o n .
logprob = models.loglogit(V, av , CHOICE)

# Create t h e Biogeme o b j e c t
the_biogeme = bio.BIOGEME( database , logprob )
the_biogeme.modelName = 'modele_0'

#??? cette fonction ne marche pas
# C a l c u l a t e  t h e  n u l l  l o g l i k e l i h o o d  f o r  r e p o r t i n g .
#the_biogeme.calculateNullLogLikelihood( av )

# E s t i m a t e  t h e  p a r a m e t e r s
results = the_biogeme.estimate()
print( results.short_summary() )

# Get t h e  r e s u l t s  i n  a pandas t a b l e
pandas_results = results.getEstimatedParameters()
print(pandas_results)

#print(database)
#print(V)


Results for model modele_0
Nbr of parameters:		5
Sample size:			5000
Excluded data:			0
Final log likelihood:		-5660.75
Akaike Information Criterion:	11331.5
Bayesian Information Criterion:	11364.09

               Value  Rob. Std err  Rob. t-test  Rob. p-value
B0_cycling -2.846748      0.093962   -30.296926  0.000000e+00
B0_driving -0.918392      0.055744   -16.475137  0.000000e+00
B0_publict -0.192053      0.037328    -5.145061  2.674332e-07
B1         -0.022493      0.010794    -2.083883  3.717082e-02
B2         -0.070829      0.037479    -1.889828  5.878101e-02


In [3]:
#modele 1
#pour Beta seul le dernier argument est important : 
#a flag that indicates if the parameter must be estimated (0) or if it keeps its default value (1).
B0_driving = Beta('B0_driving', 0, None, None, 0)
B0_cycling = Beta('B0_cycling', 0, None, None, 0)
B0_walking = Beta('B0_walking', 0, None, None, 1) # normalized to zero
B0_publict = Beta('B0_publict', 0, None, None, 0)
B1 = Beta('B1', 0, None, None, 0)
B2 = Beta('B2', 0, None, None, 0)

B1_driving = Beta('B1_driving', 0, None, None, 0)
B2_driving = Beta('B2_driving', 0, None, None, 0)

# D e f i n i t i o n  o f  t h e  u t i l i t y  f u n c t i o n s

V_driving = ( B0_driving + (B1 + B1_driving) * cost_driving + (B2 + B2_driving) * tt_driving ) #verifier ca
V_cycling = ( B0_cycling + B1 * cost_cycling + B2 * tt_cycling )
V_publict = ( B0_walking + B1 * cost_walking + B2 * tt_walking )
V_walking = ( B0_publict + B1 * cost_publict + B2 * tt_publict )


# A s s o c i a t e  u t i l i t y  f u n c t i o n s  w i t h  t h e numbering o f  a l t e r n a t i v e s

V = {1: V_driving , 2: V_cycling , 3: V_walking , 4: V_publict }

#il faut decrire le fait que tous les choix sont available
av = {1 : un, 2 : un, 3 : un, 4 : un }

# D e f i n i t i o n  o f  t h e model . This i s  t h e  c o n t r i b u t i o n  o f each
# o b s e r v a t i o n  t o  t h e  l o g l i k e l i h o o d  f u n c t i o n .
logprob = models.loglogit(V, av , CHOICE)

# Create t h e Biogeme o b j e c t
the_biogeme = bio.BIOGEME( database , logprob )
the_biogeme.modelName = 'modele_1'

#??? cette fonction ne marche pas
# C a l c u l a t e  t h e  n u l l  l o g l i k e l i h o o d  f o r  r e p o r t i n g .
#the_biogeme.calculateNullLogLikelihood( av )

# E s t i m a t e  t h e  p a r a m e t e r s
results = the_biogeme.estimate()
print( results.short_summary() )

# Get t h e  r e s u l t s  i n  a pandas t a b l e
pandas_results = results.getEstimatedParameters()
print(pandas_results)


Results for model modele_1
Nbr of parameters:		7
Sample size:			5000
Excluded data:			0
Final log likelihood:		-4780.614
Akaike Information Criterion:	9575.228
Bayesian Information Criterion:	9620.848

                Value  Rob. Std err  Rob. t-test  Rob. p-value
B0_cycling  -3.179944      0.100770   -31.556519  0.000000e+00
B0_driving   1.928358      0.111191    17.342737  0.000000e+00
B0_publict  -0.546720      0.049385   -11.070558  0.000000e+00
B1           0.011100      0.023025     0.482089  6.297426e-01
B1_driving   0.198428      0.035129     5.648523  1.618321e-08
B2          -0.457082      0.049540    -9.226448  0.000000e+00
B2_driving -22.631895      0.997203   -22.695384  0.000000e+00


statistical test to compare modele 0 and modele 1
t test i guess

B1 not significant ? but all others yes ?


In [4]:
#modele 1 again for public transport
#pour Beta seul le dernier argument est important : 
#a flag that indicates if the parameter must be estimated (0) or if it keeps its default value (1).
B0_driving = Beta('B0_driving', 0, None, None, 0)
B0_cycling = Beta('B0_cycling', 0, None, None, 0)
B0_walking = Beta('B0_walking', 0, None, None, 1) # normalized to zero
B0_publict = Beta('B0_publict', 0, None, None, 0)
B1 = Beta('B1', 0, None, None, 0)
B2 = Beta('B2', 0, None, None, 0)

B1_publict = Beta('B1_publict', 0, None, None, 0)
B2_publict = Beta('B2_publict', 0, None, None, 0)

# D e f i n i t i o n  o f  t h e  u t i l i t y  f u n c t i o n s

V_driving = ( B0_driving + (B1) * cost_driving + (B2) * tt_driving ) #verifier ca
V_cycling = ( B0_cycling + (B1 + B1_publict) * cost_cycling + (B2 + B2_publict) * tt_cycling )
V_publict = ( B0_walking + B1 * cost_walking + B2 * tt_walking )
V_walking = ( B0_publict + B1 * cost_publict + B2 * tt_publict )


# A s s o c i a t e  u t i l i t y  f u n c t i o n s  w i t h  t h e numbering o f  a l t e r n a t i v e s

V = {1: V_driving , 2: V_cycling , 3: V_walking , 4: V_publict }

#il faut decrire le fait que tous les choix sont available
av = {1 : un, 2 : un, 3 : un, 4 : un }

# D e f i n i t i o n  o f  t h e model . This i s  t h e  c o n t r i b u t i o n  o f each
# o b s e r v a t i o n  t o  t h e  l o g l i k e l i h o o d  f u n c t i o n .
logprob = models.loglogit(V, av , CHOICE)

# Create t h e Biogeme o b j e c t
the_biogeme = bio.BIOGEME( database , logprob )
the_biogeme.modelName = 'modele_1_bis'

#??? cette fonction ne marche pas
# C a l c u l a t e  t h e  n u l l  l o g l i k e l i h o o d  f o r  r e p o r t i n g .
#the_biogeme.calculateNullLogLikelihood( av )

# E s t i m a t e  t h e  p a r a m e t e r s
results = the_biogeme.estimate()
print( results.short_summary() )

# Get t h e  r e s u l t s  i n  a pandas t a b l e
pandas_results = results.getEstimatedParameters()
print(pandas_results)


Results for model modele_1_bis
Nbr of parameters:		7
Sample size:			5000
Excluded data:			0
Final log likelihood:		-5659.031
Akaike Information Criterion:	11332.06
Bayesian Information Criterion:	11377.68

               Value  Rob. Std err   Rob. t-test  Rob. p-value
B0_cycling -2.677126  1.133747e-01 -2.361307e+01  0.000000e+00
B0_driving -0.921811  5.592360e-02 -1.648341e+01  0.000000e+00
B0_publict -0.194376  3.738426e-02 -5.199410e+00  1.999219e-07
B1         -0.024989  1.103582e-02 -2.264323e+00  2.355426e-02
B1_publict -0.512029  5.164770e-16 -9.913888e+14  0.000000e+00
B2         -0.079954  3.806872e-02 -2.100250e+00  3.570686e-02
B2_publict -0.512091  2.178240e-01 -2.350939e+00  1.872612e-02


conclusion of modele 1 is that public transit does not care about money

model pref will be model 1 with extra beta 1 for driving 

model 2 socoioeconomic characteristic candidates 

- faretype : proxy for age + disabled

- age

- female

- driving_licence

-  car_ownership

- distance

candidates for my 2 interactions

- driving_licence or car_ownership : B0_driving

- car_ownership : B0_publict

lol : 

- distance : B2_walking 

- distance : B2_cycling

- driving_licence : car_ownership

- faretype : age

- faretype : B2_publict or driving

In [5]:
#modele 2 female

B0_cycling = Beta('B0_cycling', 0, None, None, 0)
B0_walking = Beta('B0_walking', 0, None, None, 1) # normalized to zero
B0_publict = Beta('B0_publict', 0, None, None, 0)
B1 = Beta('B1', 0, None, None, 0)
B2 = Beta('B2', 0, None, None, 0)

B0_cycling_f = Beta('B0_cycling_f', 0, None, None, 0)
B0_walking_f = Beta('B0_walking_f', 0, None, None, 0)
B0_publict_f = Beta('B0_publict_f', 0, None, None, 0)
B1_f = Beta('B1_f', 0, None, None, 0)
B2_f = Beta('B2_f', 0, None, None, 0)

B0_driving = Beta('B0_driving', 0, None, None, 0)
B1_driving = Beta('B1_driving', 0, None, None, 0)
B2_driving = Beta('B2_driving', 0, None, None, 0)

B0_driving_f = Beta('B0_driving_f', 0, None, None, 0)
B1_driving_f = Beta('B1_driving_f', 0, None, None, 0)
B2_driving_f = Beta('B2_driving_f', 0, None, None, 0)

# D e f i n i t i o n  o f  t h e  u t i l i t y  f u n c t i o n s

V_driving = ( (B0_driving + B0_driving_f * female) 
                + (B1 + B1_driving + B1_driving_f * female) * cost_driving 
                + (B2 + B2_driving + B2_driving_f * female) * tt_driving ) #verifier ca

V_cycling = ( (B0_cycling + B0_cycling_f * female) + (B1 + B1_f * female) * cost_cycling + (B2 + B2_f * female) * tt_cycling )
V_publict = ( (B0_walking + B0_walking_f * female) + (B1 + B1_f * female) * cost_walking + (B2 + B2_f * female) * tt_walking )
V_walking = ( (B0_publict + B0_publict_f * female) + (B1 + B1_f * female) * cost_publict + (B2 + B2_f * female) * tt_publict )

# A s s o c i a t e  u t i l i t y  f u n c t i o n s  w i t h  t h e numbering o f  a l t e r n a t i v e s

V = {1: V_driving , 2: V_cycling , 3: V_walking , 4: V_publict }

#il faut decrire le fait que tous les choix sont available
av = {1 : un, 2 : un, 3 : un, 4 : un }

# D e f i n i t i o n  o f  t h e model . This i s  t h e  c o n t r i b u t i o n  o f each
# o b s e r v a t i o n  t o  t h e  l o g l i k e l i h o o d  f u n c t i o n .
logprob = models.loglogit(V, av , CHOICE)

# Create t h e Biogeme o b j e c t
the_biogeme = bio.BIOGEME( database , logprob )
the_biogeme.modelName = 'modele_2_female'

#??? cette fonction ne marche pas
# C a l c u l a t e  t h e  n u l l  l o g l i k e l i h o o d  f o r  r e p o r t i n g .
#the_biogeme.calculateNullLogLikelihood( av )

# E s t i m a t e  t h e  p a r a m e t e r s
results = the_biogeme.estimate()
print( results.short_summary() )

# Get t h e  r e s u l t s  i n  a pandas t a b l e
pandas_results = results.getEstimatedParameters()
print(pandas_results)


Results for model modele_2_female
Nbr of parameters:		15
Sample size:			5000
Excluded data:			0
Final log likelihood:		-4755.152
Akaike Information Criterion:	9540.303
Bayesian Information Criterion:	9638.061

                  Value  Rob. Std err  Rob. t-test  Rob. p-value
B0_cycling    -2.688262      0.124202   -21.644186  0.000000e+00
B0_cycling_f  -0.421976      0.172729    -2.442992  1.456605e-02
B0_driving     1.786182      0.157273    11.357218  0.000000e+00
B0_driving_f   1.091122      0.171347     6.367917  1.916132e-10
B0_publict    -0.636033      0.072434    -8.780849  0.000000e+00
B0_publict_f   0.992540      0.094896    10.459252  0.000000e+00
B0_walking_f   0.838314      0.090868     9.225625  0.000000e+00
B1             0.053970      0.031703     1.702367  8.868660e-02
B1_driving     0.150749      0.047085     3.201621  1.366568e-03
B1_driving_f   0.009745      0.053325     0.182747  8.549963e-01
B1_f          -0.080654      0.046338    -1.740578  8.175763e-02
B2        

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf9257be-d3fc-4cee-a6ca-c5176db4a842' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

interactions non significatives : 
B2_driving_f
B1_f          pas vraiment
B1_driving_f

significatives :
B0_cycling_f  
B0_driving_f   
B0_publict_f   
B0_walking_f   

conclusions :
gender changes the base willingness to choose a mode of transport and the value of money but not really the value of time


In [6]:
#modele 2 female and distance 

B0_cycling = Beta('B0_cycling', 0, None, None, 0)
B0_walking = Beta('B0_walking', 0, None, None, 1) # normalized to zero
B0_publict = Beta('B0_publict', 0, None, None, 0)
B1 = Beta('B1', 0, None, None, 0)
B2 = Beta('B2', 0, None, None, 0)

B0_driving = Beta('B0_driving', 0, None, None, 0)
B1_driving = Beta('B1_driving', 0, None, None, 0)
B2_driving = Beta('B2_driving', 0, None, None, 0)

B0_driving_f = Beta('B0_driving_f', 0, None, None, 0)
B0_cycling_f = Beta('B0_cycling_f', 0, None, None, 0)
B0_walking_f = Beta('B0_walking_f', 0, None, None, 0)
B0_publict_f = Beta('B0_publict_f', 0, None, None, 0)
B1_f = Beta('B1_f', 0, None, None, 0)
B2_f = Beta('B2_f', 0, None, None, 0)

B0_driving_dist = Beta('B0_driving_dist', 0, None, None, 0)
B0_cycling_dist = Beta('B0_cycling_dist', 0, None, None, 0)
B0_walking_dist = Beta('B0_walking_dist', 0, None, None, 0)
B0_publict_dist = Beta('B0_publict_dist', 0, None, None, 0)

# D e f i n i t i o n  o f  t h e  u t i l i t y  f u n c t i o n s

V_driving = ( (B0_driving + B0_driving_f * female + B0_driving_dist * distance) 
               + (B1 + B1_driving + B1_driving_f * female) * cost_driving 
               + (B2 + B2_driving + B2_driving_f * female) * tt_driving ) #verifier ca

V_cycling = ( (B0_cycling + B0_cycling_f * female + B0_cycling_dist * distance) 
             + (B1 + B1_f * female) * cost_cycling 
             + (B2 + B2_f * female) * tt_cycling )

V_publict = ( (B0_walking + B0_walking_f * female + B0_publict_dist * distance) 
             + (B1 + B1_f * female) * cost_walking 
             + (B2 + B2_f * female) * tt_walking )

V_walking = ( (B0_publict + B0_publict_f * female + B0_walking_dist * distance) 
             + (B1 + B1_f * female) * cost_publict 
             + (B2 + B2_f * female) * tt_publict )

# A s s o c i a t e  u t i l i t y  f u n c t i o n s  w i t h  t h e numbering o f  a l t e r n a t i v e s

V = {1: V_driving , 2: V_cycling , 3: V_walking , 4: V_publict }

#il faut decrire le fait que tous les choix sont available
av = {1 : un, 2 : un, 3 : un, 4 : un }

# D e f i n i t i o n  o f  t h e model . This i s  t h e  c o n t r i b u t i o n  o f each
# o b s e r v a t i o n  t o  t h e  l o g l i k e l i h o o d  f u n c t i o n .
logprob = models.loglogit(V, av , CHOICE)

# Create t h e Biogeme o b j e c t
the_biogeme = bio.BIOGEME( database , logprob )
the_biogeme.modelName = 'modele_2_female'

#??? cette fonction ne marche pas
# C a l c u l a t e  t h e  n u l l  l o g l i k e l i h o o d  f o r  r e p o r t i n g .
#the_biogeme.calculateNullLogLikelihood( av )

# E s t i m a t e  t h e  p a r a m e t e r s
results = the_biogeme.estimate()
print( results.short_summary() )

# Get t h e  r e s u l t s  i n  a pandas t a b l e
pandas_results = results.getEstimatedParameters()
print(pandas_results)


Results for model modele_2_female
Nbr of parameters:		19
Sample size:			5000
Excluded data:			0
Final log likelihood:		-4531.728
Akaike Information Criterion:	9101.456
Bayesian Information Criterion:	9225.282

                    Value  Rob. Std err  Rob. t-test  Rob. p-value
B0_cycling      -2.077668      0.144200   -14.408187  0.000000e+00
B0_cycling_dist  0.000379      0.000037    10.173685  0.000000e+00
B0_cycling_f    -0.417238      0.164735    -2.532778  1.131627e-02
B0_driving       2.043407      0.173140    11.802047  0.000000e+00
B0_driving_dist -0.001563      0.000099   -15.727667  0.000000e+00
B0_driving_f     1.005462      0.179037     5.615941  1.954949e-08
B0_publict      -0.498722      0.085626    -5.824408  5.731549e-09
B0_publict_dist  0.000667      0.000046    14.356215  0.000000e+00
B0_publict_f     1.021709      0.096113    10.630300  0.000000e+00
B0_walking_dist  0.000517      0.000033    15.562575  0.000000e+00
B0_walking_f     0.890068      0.096049     9.266769 

variables non significatives :
B0_driving_f     
B1              
B1_driving       
B1_driving_f     
B1_f            
B2_driving       
B2_driving_f    
B2_f 


les 4 interactions de distance : intercept sont significatives :

B0_cycling_dist 
B0_driving_dist 
B0_publict_dist  
B0_walking_dist  